In [ ]:
import torch

#### 3-1. 변수 정의
- 파이토치를 활용해 코드로도 확인해 보겠습니다. 우선 입력 벡터 시퀀스 𝐗 와 쿼리, 키, 밸류 구축에 필요한 행렬들을 앞선 예시 그대로 정의합니다. 코드1과 같습니다.

In [ ]:
x = torch.tensor([
  [1.0, 0.0, 1.0, 0.0],
  [0.0, 2.0, 0.0, 2.0],
  [1.0, 1.0, 1.0, 1.0],
])
w_query = torch.tensor([
  [1.0, 0.0, 1.0],
  [1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0],
  [0.0, 1.0, 1.0]
])
w_key = torch.tensor([
  [0.0, 0.0, 1.0],
  [1.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [1.0, 1.0, 0.0]
])
w_value = torch.tensor([
  [0.0, 2.0, 0.0],
  [0.0, 3.0, 0.0],
  [1.0, 0.0, 3.0],
  [1.0, 1.0, 0.0]
])

#### 3-2.  Query, Key, Value 만들기
- 코드2는 수식2를 계산해 입력 벡터 시퀀스로 쿼리, 키, 밸류 벡터들을 만드는 파트입니다. torch.matmul는 행렬곱을 수행하는 함수입니다.

In [ ]:
keys = torch.matmul(x, w_key)
querys = torch.matmul(x, w_query)
values = torch.matmul(x, w_value)

#### 3-3. Attention Score 만들기
- 코드3은 코드2에서 만든 쿼리와 키 벡터들을 행렬곱해서 어텐션 스코어를 만드는 과정입니다. keys.T는 키 벡터들을 전치한 행렬입니다.

In [ ]:
attn_scores = torch.matmul(querys, keys.T)
attn_scores

tensor([[ 2.,  4.,  4.],
        [ 4., 16., 12.],
        [ 4., 12., 10.]])

#### 3-4. 소프트맥스 확률값 만들기
- 코드4는 코드3의 결과에 키 벡터의 차원수의 제곱근으로 나눠준 뒤 소프트맥스를 취하는 과정입니다.

In [ ]:
import numpy as np
from torch.nn.functional import softmax
key_dim_sqrt = np.sqrt(keys.shape[-1])
attn_scores_softmax = softmax(attn_scores / key_dim_sqrt, dim=-1)

In [ ]:
attn_scores_softmax

tensor([[1.3613e-01, 4.3194e-01, 4.3194e-01],
        [8.9045e-04, 9.0884e-01, 9.0267e-02],
        [7.4449e-03, 7.5471e-01, 2.3785e-01]])

#### 3-5. 소프트맥스 확률과 밸류를 가중합하기
- 코드5는 코드4에서 구한 소프트맥스 확률과 밸류 벡터들을 가중합하는 과정을 수행한다.
- 셀프 어텐션의 학습 대상은 쿼리, 키, 밸류를 만드는 가중치 행렬입니다. 코드 예시에서는 w_query, w_key, w_value입니다. 이들은 태스크(예: 기계 번역)를 가장 잘 수행하는 방향으로 학습 과정에서 업데이트됩니다.

In [ ]:
weighted_values = torch.matmul(attn_scores_softmax, values)
weighted_values

tensor([[1.8639, 6.3194, 1.7042],
        [1.9991, 7.8141, 0.2735],
        [1.9926, 7.4796, 0.7359]])